In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
# Каталог с данными для обучения
train_dir = 'data/dogs_vs_cats/train'
# Каталог с данными для проверки
val_dir = 'data/dogs_vs_cats/val'
# Каталог с данными для тестирования
test_dir = 'data/dogs_vs_cats/test'

# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

## Создаем сверточную нейронную сеть
**Архитектура сети**

Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
Слой подвыборки, выбор максимального значения из квадрата 2х2

Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
Слой подвыборки, выбор максимального значения из квадрата 2х2

Слой свертки, размер ядра 3х3, количество карт признаков - 64 шт., функция активации ReLU.
Слой подвыборки, выбор максимального значения из квадрата 2х2

Слой преобразования из двумерного в одномерное представление

Полносвязный слой, 64 нейрона, функция активации ReLU.
Слой Dropout.

Выходной слой, 1 нейрон, функция активации sigmoid
Слои с 1 по 6 используются для выделения важных признаков в изображении, а слои с 7 по 10 - для классификации.

In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [4]:
model.compile(loss=['binary_crossentropy'],
             optimizer='adam',
             metrics=['accuracy'])

### Создаем генератор изображений
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [5]:
datagen = ImageDataGenerator(rescale=1. / 255)

Генератор данных для обучения на основе изображений из каталога

In [6]:
train_generator = datagen.flow_from_directory(
    train_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary') # or categorical

Found 17500 images belonging to 2 classes.


In [7]:
test_generator = datagen.flow_from_directory(
    test_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary') # or categorical

Found 3750 images belonging to 2 classes.


In [8]:
valid_generator = datagen.flow_from_directory(
    val_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary') # or categorical

Found 3750 images belonging to 2 classes.


## Обучаем модель с использованием генераторов
train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
1093/1093 [==============================] - 164s 150ms/step - loss: 0.6232 - acc: 0.6463 - val_loss: 0.5291 - val_acc: 0.7353
Epoch 2/10
1093/1093 [==============================] - 161s 147ms/step - loss: 0.5337 - acc: 0.7302 - val_loss: 0.5021 - val_acc: 0.7544
Epoch 3/10
1093/1093 [==============================] - 161s 147ms/step - loss: 0.4760 - acc: 0.7711 - val_loss: 0.4618 - val_acc: 0.7804
Epoch 4/10
1093/1093 [==============================] - 161s 148ms/step - loss: 0.4289 - acc: 0.8053 - val_loss: 0.4292 - val_acc: 0.7938
Epoch 5/10
1093/1093 [==============================] - 161s 148ms/step - loss: 0.3820 - acc: 0.8303 - val_loss: 0.4016 - val_acc: 0.8200
Epoch 6/10
1093/1093 [==============================] - 162s 148ms/step - loss: 0.3305 - acc: 0.8614 - val_loss: 0.4144 - val_acc: 0.8141
Epoch 7/10
1093/1093 [==============================] - 161s 147ms/step - loss: 0.2866 - acc: 0.8783 - val_loss: 0.3908 - val_acc: 0.8345
Epoch 8/10
1093/1093 [============

In [10]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 83.87%


In [11]:
model_json = model.to_json()
with open("Results/dogs_vs_cats_model.json", "w") as f:
    f.write(model_json)

model.save_weights("Results/dogs_vs_cats_model.h5")